In [1]:
!gsutil cp gs://khyatis-bucket/dataproc_ml_python-0.1.0-py3-none-any.whl .

Copying gs://khyatis-bucket/dataproc_ml_python-0.1.0-py3-none-any.whl...
/ [1 files][ 23.6 KiB/ 23.6 KiB]                                                
Operation completed over 1 objects/23.6 KiB.                                     


In [2]:
!pip install dataproc_ml_python-0.1.0-py3-none-any.whl

Processing /dataproc_ml_python-0.1.0-py3-none-any.whl


dataproc-ml-python is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [3]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, ArrayType
from pyspark.sql.functions import col, array
from google.cloud.dataproc.ml.inference import PyTorchModelHandler

2025-08-28 08:45:43.402270: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-28 08:45:43.601400: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756370743.628923 1568893 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756370743.646526 1568893 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-28 08:45:43.889589: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
spark = SparkSession.builder.appName("PyTorchInferenceExample").getOrCreate()

25/08/28 08:45:58 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
input_df = spark.read.csv("gs://khyatis-bucket/model_input_data_1000.csv", header=True, inferSchema=True)
input_df.show()


25/08/28 08:47:05 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/08/28 08:47:20 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/08/28 08:47:35 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/08/28 08:47:50 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/08/28 08:48:05 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/08/28 08:48:20 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registere

+--------------------+-------------------+
|                col1|               col2|
+--------------------+-------------------+
|  -10.49550904126364| 15.793832837361286|
| -15.639082040102982|-14.918121713742238|
|   2.334217525875596|   9.03901072766134|
| -13.158239600964093|-12.109398071716646|
|   3.562815254383147|-12.176518727975447|
|  1.5014570897475108| 16.664243678792083|
|   12.32420841596247| -8.696853220078959|
|   -8.42547489497358| -3.507978354334571|
|    2.10711182104013|-10.365064534150168|
| -16.597036748870828|  7.766465614083062|
|   2.555101629807392|  5.780722412036344|
|  0.7300048334520057|-10.324780363413398|
| -10.163061099934389|-6.0747040079893155|
| -14.270822937048223|  7.062699526326531|
|   4.346189443793728| 21.290691053437293|
|  -16.60334381574049|-2.0091437459760457|
| -2.7004378249636423|-14.898887225501374|
|-0.04889526754601949|  7.322118790184925|
|   3.424074568226829| -11.65563012147405|
|  -21.52914384938552|  7.335592828060849|
+----------

In [7]:
preprocessed_df = input_df.withColumn("features", array(col("col1"), col("col2")))
preprocessed_df.show()

+--------------------+-------------------+--------------------+
|                col1|               col2|            features|
+--------------------+-------------------+--------------------+
|  -10.49550904126364| 15.793832837361286|[-10.495509041263...|
| -15.639082040102982|-14.918121713742238|[-15.639082040102...|
|   2.334217525875596|   9.03901072766134|[2.33421752587559...|
| -13.158239600964093|-12.109398071716646|[-13.158239600964...|
|   3.562815254383147|-12.176518727975447|[3.56281525438314...|
|  1.5014570897475108| 16.664243678792083|[1.50145708974751...|
|   12.32420841596247| -8.696853220078959|[12.3242084159624...|
|   -8.42547489497358| -3.507978354334571|[-8.4254748949735...|
|    2.10711182104013|-10.365064534150168|[2.10711182104013...|
| -16.597036748870828|  7.766465614083062|[-16.597036748870...|
|   2.555101629807392|  5.780722412036344|[2.55510162980739...|
|  0.7300048334520057|-10.324780363413398|[0.73000483345200...|
| -10.163061099934389|-6.074704007989315

In [8]:
handler = PyTorchModelHandler().model_path("gs://khyatis-bucket/pytorch_model_two_columns_demo.pth").input_col("features").set_return_type(ArrayType(DoubleType()))
handler.transform(preprocessed_df).show()

+--------------------+-------------------+--------------------+--------------------+
|                col1|               col2|            features|         predictions|
+--------------------+-------------------+--------------------+--------------------+
|  -10.49550904126364| 15.793832837361286|[-10.495509041263...|[10.364126205444336]|
| -15.639082040102982|-14.918121713742238|[-15.639082040102...|[-61.430030822753...|
|   2.334217525875596|   9.03901072766134|[2.33421752587559...|[22.327882766723633]|
| -13.158239600964093|-12.109398071716646|[-13.158239600964...|[-50.867462158203...|
|   3.562815254383147|-12.176518727975447|[3.56281525438314...|[-17.76085090637207]|
|  1.5014570897475108| 16.664243678792083|[1.50145708974751...| [35.95881652832031]|
|   12.32420841596247| -8.696853220078959|[12.3242084159624...| [6.632394790649414]|
|   -8.42547489497358| -3.507978354334571|[-8.4254748949735...|[-24.215381622314...|
|    2.10711182104013|-10.365064534150168|[2.10711182104013...| [